In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM2
from utils import read_plan, read_json_file
from evaluator import ExecutabilityEvaluator
from collections import defaultdict

In [2]:
def test_exe(domain_name, trace, test_trace=None):
    debug = {
        # 'get_transition_sets': True,
        # 'extract_holes': True,
        # 'find_holes': True,
    }
    try:
        locm2 = LOCM2(state_param=True, viz=False, debug=debug)
        model = locm2.extract_model([trace])
        
        
        pddl_domain = model.to_pddl_domain("test_bw")
        action_trace = [step.action for step in trace]

        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            exe = evaluator.get_overall_executability('l', test_trace, set(), set())
        else:
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [3]:
def test_acceptance(domain_name,train_traces, test_traces):
    debug = {}
    try:
        locm = LOCM2(state_param=True, viz=False, debug=debug)
        model = locm.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)

        evaluator = ExecutabilityEvaluator(learned_domain, debug=True)
        exe = evaluator.get_acceptance_rate(test_traces, test_traces)

        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1

In [4]:
def test_cross_exe(domain_name, po_trace, to_trace):
    debug = {}
    try:
        polocm2 = LOCM2(state_param=True, viz=False, debug=debug)
        model = polocm2.extract_model([po_trace])
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        evaluator = ExecutabilityEvaluator(learned_domain, 'cross',gt_filename, debug=True)
        action_trace = [step.action for step in to_trace]
        exe = evaluator.get_cross_executability(action_trace)
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [5]:
train_traces = defaultdict(list)
data = read_json_file("../../data/training_data/traces_plan_po_r10.json")
for learning_obj in data:
    domain_name = learning_obj["domain"]
    train_traces[domain_name].append(learning_obj)

In [6]:
with open("./locm2_self_acceptance_rate.csv", "w") as f:
    f.write("ID, Domain, len, len%, valid_exe, invalid_exe\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]

            try:
                acceptance_rate = test_acceptance(domain, traces, traces)
            except Exception as e:
                print(f"Error in balanced executability for domain {domain}: {e}")
                continue

        # Test balanced executability
            f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {acceptance_rate[0]}, {acceptance_rate[1]}\n")
            f.flush()
            print(f"Balanced Executability for {domain}: {acceptance_rate}")

Testing domain: blocksworld
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blocksworld: (0, 1)
Error processing domain blocksworld: 'Trace' object has no attribute 'name'
Balanced Executability for blockswo

KeyboardInterrupt: 

In [18]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)

test_plan2 = "(pickup b2?object),(stack b2?object b1?object),(unstack b3?object b4?object),(putdown b3?object)"
test_trace2 = read_plan(test_plan)
po_trace2 = test_trace.to_partial_ordered_trace(0.3)

# test_exe("plan1", trace)
test_acceptance("plan2", [test_trace, test_trace2], [test_trace])

[          pickup.0  stack.0
pickup.0         0        1
stack.0          1        0,           pickup.1  stack.1  stack.2
pickup.1         0        1        0
stack.1          0        0        1
stack.2          0        0        0]
Error processing domain plan2: 'bool' object is not subscriptable


(0, 1)

In [19]:


# test_exe("plan1", trace)
# test_cross_exe("blocksworld", test_trace, test_trace)
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

In [20]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         details = line.split("&&")
#         name = f"{details[0]}-{details[2]}-{details[3]}"
      
#         plan = details[-1]
#         trace = read_plan(plan)
#         test_exe(name, trace)

In [21]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     with open("res.csv", "w") as out:
#         lines = f.readlines()
#         for line in lines:
#             try:
#                 details = line.split("&&")
#                 name = f"{details[0]}-{details[2]}-{details[3]}"
            
#                 plan = details[-1]
#                 trace = read_plan(plan)
           

#                 domain_name = details[0]
            
#                 exe_l, exe_gt = test_cross_exe(domain_name, trace, trace)
#             except Exception as e:
#                 print(f"Error processing task {name}: {e}")
#                 exe_l, exe_gt = 0, 0
#             print(f"Executability for {name}: {exe_l}, {exe_gt}")
#             out.write(f"{name},{exe_l}, {exe_gt}\n")
#             out.flush()